In [1]:
import numpy as np
import pandas as pd
import matplotlib
import sklearn
import math
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.svm import SVC
import functools
from copy import deepcopy
import math

In [2]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
np.set_printoptions(precision=4, suppress=True)

In [3]:
x = '..\Problem\shuffled-full-set-hashed.csv'
data = pd.read_csv(x, header=None)
df = pd.DataFrame(data)
df.columns = ['output','input']
df.dropna(inplace=True)

In [4]:
# WordMapper
mapper = {}
ID = 0
for i in range(len(df)):
    for word in df.iloc[i][1].split(' '):
        if word not in mapper:
            mapper[word] = (word, ID)
            ID += 1

In [5]:
# WordSwapperWithMapper
temp = []
x = []
for i in range(len(df)):
    for word in df.iloc[i][1].split(' '):
        temp.append(mapper[word][1])
    df.iloc[i][1] = temp[:]
    temp = []

In [6]:
# Output Dic {Output: [Input(s)]}
docForEach = {}
for i in range(len(df)):
    if df.iloc[i][0] in docForEach:
        docForEach[df.iloc[i][0]].extend(df.iloc[i][1][:])
    else:
        docForEach[df.iloc[i][0]] = df.iloc[i][1][:]
keys = docForEach.keys()
lenForEach = {}        
temp = {}
for key in docForEach:
    lenForEach[key] = len(docForEach[key])
    for word in docForEach[key]:
        if word in temp:
            temp[word] += 1
        else:
            temp[word] = 1
    docForEach[key] = deepcopy(temp)
    temp={}

In [7]:
docForAll = {key: {i: 0 for i in range(len(mapper))} for key in docForEach.keys()}

In [19]:
len(mapper)

1037934

In [8]:
for key in docForAll:
    for word in docForAll[key]:
        if word in docForEach[key]:
            docForAll[key][word] = 1

In [9]:
DataFrame = pd.DataFrame(data=[docForAll[key] for key in keys], index=keys)

In [10]:
DataFrame

0        1        2        3        4        5        \
DELETION OF INTEREST           1        1        1        1        1        1   
RETURNED CHECK                 1        0        1        1        1        1   
BILL                           1        1        1        1        1        1   
POLICY CHANGE                  1        1        1        1        1        1   
CANCELLATION NOTICE            1        1        1        1        1        1   
DECLARATION                    1        1        1        1        1        1   
CHANGE ENDORSEMENT             1        1        0        1        1        1   
NON-RENEWAL NOTICE             1        0        1        1        0        1   
BINDER                         1        1        1        1        1        1   
REINSTATEMENT NOTICE           1        0        1        0        1        1   
EXPIRATION NOTICE              1        0        1        1        1        1   
INTENT TO CANCEL NOTICE        1        0        1        1        1        1   
APPLICATION                    1        1        1        1        1        1   
BILL BINDER                    1        0        1        1        1        1   

                         6        7        8        9        10       11       \
DELETION OF INTEREST           1        1        1        1        1        1   
RETURNED CHECK                 1        1        1        1        1        1   
BILL                           1        1        1        1        1        1   
POLICY CHANGE                  1        1        1        1        1        1   
CANCELLATION NOTICE            0        1        1        1        1        1   
DECLARATION                    1        1        1        1        1        1   
CHANGE ENDORSEMENT             0        0        1        1        1        1   
NON-RENEWAL NOTICE             0        1        1        1        1        1   
BINDER                         1        1        1        1        1        1   
REINSTATEMENT NOTICE           0        1        1        1        1        1   
EXPIRATION NOTICE              0        1        1        0        1        1   
INTENT TO CANCEL NOTICE        0        1        1        0        1        1   
APPLICATION                    0        0        1        1        1        1   
BILL BINDER                    0        1        1        0        1        1   

                         12       13       14       15       16       17       \
DELETION OF INTEREST           1        1        1        1        1        1   
RETURNED CHECK                 1        1        1        1        1        1   
BILL                           1        1        1        1        1        1   
POLICY CHANGE                  1        1        1        1        1        1   
CANCELLATION NOTICE            1        1        1        1        1        1   
DECLARATION                    1        1        1        1        1        1   
CHANGE ENDORSEMENT             0        1        1        1        1        0   
NON-RENEWAL NOTICE             0        1        1        1        1        0   
BINDER                         0        1        1        1        1        1   
REINSTATEMENT NOTICE           1        1        1        1        1        0   
EXPIRATION NOTICE              1        1        0        1        1        0   
INTENT TO CANCEL NOTICE        0        1        1        1        1        0   
APPLICATION                    1        1        1        1        1        1   
BILL BINDER                    0        1        1        1        1        0   

                         18       19       20       21       22       23       \
DELETION OF INTEREST           1        1        1        1        1        1   
RETURNED CHECK                 1        1        1        1        1        1   
BILL                           1        1        1        1        1        1   
POLICY CHANGE                  1        1        1   

In [11]:
def computeTF(wordDict, length):
    return {word: round(count/length, 5) for word,count in wordDict.items()}

In [12]:
tfForEach = {key: computeTF(docForEach[key],lenForEach[key]) for key in docForEach.keys()}

In [13]:
def computeIDF(docList):
    idfDict = {}
    N = len(docList)
    
    #counts the number of documents that contains word w
    idfDict = dict.fromkeys(docList[0].keys(), 0)
    for doc in docList:
        for word, val in doc.items():
            if val > 0:
                idfDict[word] +=1
                
    #divide N by denominator above, take the log of that
    for word, val in idfDict.items():
        idfDict[word]= math.log(N / float(val)) 

    return idfDict

In [14]:
idfForAllWords = computeIDF([docForAll[key] for key in keys])

In [15]:
def computeTFIDF(tf, idfs):
    tfidf = {}
    for word, val in tf.items():
        tfidf[word] = val * idfs[word]
    return tfidf

In [16]:
tfidfForEach = {key: computeTFIDF(tfForEach[key], idfForAllWords) for key in keys}

In [17]:
DataFrameTFIDF = pd.DataFrame(data=[tfidfForEach[key] for key in keys], index=keys)

In [18]:
DataFrameTFIDF

0         1             2             3        \
DELETION OF INTEREST         0.0  0.000101  0.000000e+00  0.000000e+00   
RETURNED CHECK               0.0       NaN  0.000000e+00  7.410797e-07   
BILL                         0.0  0.000000  2.223239e-06  1.482159e-06   
POLICY CHANGE                0.0  0.000006  1.482159e-06  1.111620e-05   
CANCELLATION NOTICE          0.0  0.000017  1.482159e-06  2.964319e-06   
DECLARATION                  0.0  0.000011  2.223239e-06  5.187558e-06   
CHANGE ENDORSEMENT           0.0  0.000006           NaN  1.482159e-06   
NON-RENEWAL NOTICE           0.0       NaN  3.705399e-06  7.410797e-06   
BINDER                       0.0  0.000006  1.482159e-06  4.446478e-06   
REINSTATEMENT NOTICE         0.0       NaN  7.410797e-07           NaN   
EXPIRATION NOTICE            0.0       NaN  8.892957e-06  2.223239e-06   
INTENT TO CANCEL NOTICE      0.0       NaN  5.928638e-06  7.410797e-07   
APPLICATION                  0.0  0.000000  0.000000e+00  1.185728e-05   
BILL BINDER                  0.0       NaN  7.410797e-07  4.446478e-06   

                              4        5         6         7        8        \
DELETION OF INTEREST     0.000000e+00      0.0  0.000000  0.000018      0.0   
RETURNED CHECK           8.151877e-06      0.0  0.000025  0.000005      0.0   
BILL                     1.333943e-05      0.0  0.000000  0.000011      0.0   
POLICY CHANGE            2.149131e-05      0.0  0.000059  0.000009      0.0   
CANCELLATION NOTICE      7.410797e-07      0.0       NaN  0.000074      0.0   
DECLARATION              8.151877e-06      0.0  0.000025  0.000008      0.0   
CHANGE ENDORSEMENT       4.446478e-06      0.0       NaN       NaN      0.0   
NON-RENEWAL NOTICE                NaN      0.0       NaN  0.000025      0.0   
BINDER                   1.037512e-05      0.0  0.000000  0.000002      0.0   
REINSTATEMENT NOTICE     0.000000e+00      0.0       NaN  0.000052      0.0   
EXPIRATION NOTICE        7.410797e-07      0.0       NaN  0.000082      0.0   
INTENT TO CANCEL NOTICE  7.410797e-07      0.0       NaN  0.000002      0.0   
APPLICATION              6.669717e-06      0.0       NaN       NaN      0.0   
BILL BINDER              6.669717e-05      0.0       NaN  0.000003      0.0   

                          9        10       11        12       13       \
DELETION OF INTEREST     0.000000      0.0      0.0  0.000000      0.0   
RETURNED CHECK           0.000024      0.0      0.0  0.000013      0.0   
BILL                     0.000000      0.0      0.0  0.000000      0.0   
POLICY CHANGE            0.000002      0.0      0.0  0.000013      0.0   
CANCELLATION NOTICE      0.000002      0.0      0.0  0.000000      0.0   
DECLARATION              0.000005      0.0      0.0  0.000000      0.0   
CHANGE ENDORSEMENT       0.000002      0.0      0.0       NaN      0.0   
NON-RENEWAL NOTICE       0.000002      0.0      0.0       NaN      0.0   
BINDER                   0.000000      0.0      0.0       NaN      0.0   
REINSTATEMENT NOTICE     0.000000      0.0      0.0  0.000004      0.0   
EXPIRATION NOTICE             NaN      0.0      0.0  0.000004      0.0   
INTENT TO CANCEL NOTICE       NaN      0.0      0.0       NaN      0.0   
APPLICATION              0.000002      0.0      0.0  0.000000      0.0   
BILL BINDER                   NaN      0.0      0.0       NaN      0.0   

                              14       15       16        17       18       \
DELETION OF INTEREST     0.000000e+00      0.0      0.0  0.000000      0.0   
RETURNED CHECK           2.223239e-06      0.0      0.0  0.000817      0.0   
BILL                     7.410797e-07      0.0      0.0  0.000000      0.0   
POLICY CHANGE            2.964319e-06      0.0      0.0  0.000011      0.0   
CANCELLATION NOTICE      1.482159e-06      0.0      0.0  0.000000      0.0   
DECLARATION              6.669717e-06      0.0      0.0  0.000006      0.0   
CHANGE ENDORSEMENT       2.223239e-06      0.0      0.0       NaN      